In [2]:
import os
import json
import torch
import cv2
import matplotlib.pyplot as plt
import glob
import numpy as np
import shutil
import torchvision
#import mediapipe as mp
import ultralytics
import yaml
from tqdm import tqdm
from PIL import Image
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [1]:
data_root = "D:\\project"
file_root = "D:\\project\\data"
train_root = f'{file_root}\\train'
valid_root = f'{file_root}\\valid'
test_root = f'{file_root}\\test'

In [ ]:
# json 파일에 있는 bbox 정보를 yolo 형식으로
# class_name x_center y_center w h

def cvt_bbox_yolo(img_width, img_height, bbox):
    x, y, w, h = map(float, bbox.split(', '))
    
    x_center = (x + w/2) / img_width
    y_center = (y + h/2) / img_height
    
    width = w / img_width
    height = h / img_height
    
    return x_center, y_center, width, height

def cvt_json_yolo(json_data):
    img_width, img_height = map(int, json_data['metadata']['scene_res'].split(' x '))
    
    bbox = json_data['bboxdata']['bbox_location']
    
    x_center, y_center, width, height = cvt_bbox_yolo(img_width, img_height, bbox)
    
    class_id = Pass
    
    yolo_format = f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"
    
    return yolo_format

def json_files_out(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # 모든 JSON 파일 처리
    for filename in os.listdir(input_dir):
        if filename.endswith('.json'):
            json_path = os.path.join(input_dir, filename)
            
            # JSON 파일 읽기
            with open(json_path, 'r') as f:
                json_data = json.load(f)
            
            # YOLO 형식으로 convert
            yolo_format = cvt_json_yolo(json_data)
            
            # 파일로 저장
            output_filename = json_data['metadata']['file_name'].replace('.JPG', '.txt')
            output_path = os.path.join(output_dir, output_filename)
            
            with open(output_path, 'w') as f:
                f.write(yolo_format)
            
            print('완료')

# 사용 예
input_directory = 'path/to/json/files'  # JSON 파일이 있는 디렉토리
output_directory = 'path/to/output/files'  # YOLO 형식 파일을 저장할 디렉토리

json_files_out(input_directory, output_directory)